In [1]:
import pandas as pd
import os

In [2]:
os.listdir()

['embeddings_meds.csv',
 '.DS_Store',
 'disease_cause.csv',
 'medicine_side_effects.csv',
 'Medicine_Embeddings_Clustering.ipynb',
 'medicine_uses.csv',
 'disease_specialty.csv',
 'embeddigs_diseases.csv',
 'disease_link_prediction.ipynb',
 'Medicine_Embeddings_Linkage.ipynb',
 'Medicine_Embeddings_Clustering-Copy1.ipynb',
 'diseases_embedding-Copy1.ipynb',
 '.ipynb_checkpoints',
 'application.log',
 'diseases_embedding.ipynb',
 'disease_symptoms.csv']

In [25]:
df1 = pd.read_csv('medicine_uses.csv')
df2 = pd.read_csv('medicine_side_effects.csv')

df1['subject']=df1['subject'].strip()
df1['predicate']=df1['predicate'].strip()
df1['object']=df1['object'].str

df2['subject']=df2['subject'].str.replace(" ","")
df2['predicate']=df2['predicate'].str.replace(" ","")
df2['object']=df2['object'].str.replace(" ","")

final_df=pd.concat([df1,df2])

final_df.to_csv('embeddings_meds.csv',index=False)

In [3]:
final_df = pd.read_csv('embeddings_meds.csv')

In [4]:
final_df.head()

,subject,predicate,object
0,"""calcifediol""",use,"""metabolicbonedisease"""
1,"""calcifediol""",use,"""kidneydisease"""
2,"""calcifediol""",use,"""vitaminDdeficiency"""
3,"""calcifediol""",use,"""secondaryhyperparathyroidismofrenalorigin"""
4,"""armodafinil""",use,"""narcolepsy"""


In [21]:
final_df['predicate'].unique()

array(['use', 'side_effect'], dtype=object)

In [5]:
import ampligraph
import numpy as np
from ampligraph.evaluation import train_test_split_no_unseen
from ampligraph.datasets import load_from_csv
import tensorflow as tf
from ampligraph.evaluation import evaluate_performance
from ampligraph.discovery import find_clusters
from sklearn.cluster import KMeans,DBSCAN

In [10]:
X = load_from_csv('.','embeddings_meds.csv',sep=',')
X=X[1:]
X_train, X_valid = train_test_split_no_unseen(np.array(X), test_size=int(X.shape[0]/10))
from ampligraph.latent_features import ComplEx

model = ComplEx(batches_count=50,
                epochs=300,
                k=100,
                eta=20,
                optimizer='adam', 
                optimizer_params={'lr':1e-4},
                loss='multiclass_nll',
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                seed=0, 
                verbose=True)

print('Train set size: ', X_train.shape)
print('Test set size: ', X_valid.shape)



Train set size:  (8994, 3)
Test set size:  (999, 3)


In [11]:
tf.logging.set_verbosity(tf.logging.ERROR)
model.fit(X_train)

Average Loss:   0.151160: 100%|██████████| 300/300 [03:44<00:00,  1.34epoch/s]


In [12]:
model

In [13]:
filter_triples = np.concatenate((X_train, X_valid))
ranks = evaluate_performance(X_valid,
                             model=model, 
                             filter_triples=filter_triples,
                             use_default_protocol=True,
                             verbose=True)

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 999/999 [00:06<00:00, 163.06it/s]


In [14]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mr = mr_score(ranks)
mrr = mrr_score(ranks)

print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))



MRR: 0.33
MR: 146.16
Hits@10: 0.54
Hits@3: 0.38
Hits@1: 0.23


In [15]:
meds =np.array(list(set(X_train[:,0])))
meds_embeddings = dict(zip(meds, model.get_embeddings(meds)))
len(meds)

1270

In [16]:
len(dict(zip(meds,(model.get_embeddings(meds)))))

1270

In [19]:
medicines =np.array(list(set(X_train[:,0])))
medicines = dict(zip(medicines, model.get_embeddings(medicines)))
len(medicines)

1270

In [20]:
X

array([['calcifediol', 'use', 'metabolic bone disease'],
       ['calcifediol', 'use', 'kidney disease'],
       ['calcifediol', 'use', 'vitamin D deficiency'],
       ...,
       ['Hydrochlorothiazide / Triamterene', 'side_effect', 'fatigue'],
       ['Hydrochlorothiazide / Triamterene', 'side_effect', 'dizziness'],
       ['Hydrochlorothiazide / Triamterene', 'side_effect',
        'blurred vision']], dtype=object)

In [75]:
X_unseen = np.array([
    
    ['rifaximin','side_effect','nausea'],
    ['griseofulvin','side_effect', 'diarrhea'],
    ['armodafinil','side_effect','nervousness'],
    ["azathioprine",'side_effect','fatigue']
    
    
])

In [76]:
positives_filter = X

In [77]:
unseen_filter = np.array(list({tuple(i) for i in np.vstack((positives_filter, X_unseen))}))

In [78]:
ranks_unseen = evaluate_performance(
    X_unseen, 
    model=model, 
    filter_triples=unseen_filter,
    corrupt_side = 's+o',
    use_default_protocol=False,
    verbose=True,
    filter_unseen=True
)

100%|██████████| 4/4 [00:00<00:00, 41.01it/s]


In [79]:
scores = model.predict(X_unseen)
from scipy.special import expit
probs = expit(scores)
pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      scores,
                      probs)),
             columns=['statement', 'rank', 'score', 'prob']).sort_values("score", ascending=False)

,statement,rank,score,prob
2,armodafinil side_effect nervousness,2814,-0.195489,0.451283
3,azathioprine side_effect fatigue,3896,-0.724103,0.326490
0,rifaximin side_effect nausea,4077,-2.951048,0.049687
1,griseofulvin side_effect diarrhea,5164,-4.539962,0.010561
